## Image Creation with Deep Convolutional GANs(Generative Adversarial Networks)

#### Part 1: Data Preprocessing

In [1]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
# Setting some hyperparameters
batchSize = 64 # set the size of the batch.
imageSize = 64 # set the size of the generated images (64x64).

In [3]:
# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

C:\Users\amira\Anaconda3\envs\face\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [4]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # download the training set in the ./data folder and i apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # I use dataLoader to get the images of the training set batch by batch.

Files already downloaded and verified


In [5]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
# Defining the generator

class G(nn.Module): # I introduce a class to define the generator.

    def __init__(self): # I introduce the __init__() function that will define the architecture of the generator.
        super(G, self).__init__() # I inherit from the nn.Module tools.
        self.main = nn.Sequential( # I create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), # I start with an inversed convolution.
            nn.BatchNorm2d(512), # I normalize all the features along the dimension of the batch.
            nn.ReLU(True), # I apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # I add another inversed convolution.
            nn.BatchNorm2d(256), # I normalize again.
            nn.ReLU(True), # I apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # I add another inversed convolution.
            nn.BatchNorm2d(128), # I normalize again.
            nn.ReLU(True), # I apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # I add another inversed convolution.
            nn.BatchNorm2d(64), # I normalize again.
            nn.ReLU(True), # I apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # I add another inversed convolution.
            nn.Tanh() # I apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )
